# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [79]:
import pickle

with open('train.p', 'rb') as f:
    train = pickle.load(f)
with open('test.p', 'rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'].astype('float32'), train['labels'].astype('float32')
X_test, y_test = test['features'].astype('float32'), test['labels'].astype('float32')

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [80]:
import numpy as np

X_train /= 255.0
X_train -= 0.5

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [73]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

model = Sequential()

model.add(Dense(3072, input_shape=(3072,), name='input'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3072, name='hidden1'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(43, name='output'))
model.add(Activation('softmax'))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [74]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train, 43)
Y_test = np_utils.to_categorical(y_test, 43)

X_train = X_train.reshape(X_train.shape[0], 3072)
X_test = X_test.reshape(X_test.shape[0], 3072)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, 
                    Y_train, 
                    batch_size=128, 
                    nb_epoch=2,
                    verbose=1)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

/home/mimoralea/anaconda3/envs/keras/lib/python3.5/site-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


Epoch 1/2
39209/39209 [==============================] - 4s - loss: 1.6106 - acc: 0.5570     
Epoch 2/2
39209/39209 [==============================] - 4s - loss: 0.8069 - acc: 0.7636     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [75]:
from sklearn.model_selection import train_test_split

# Split some of the training data into a validation dataset.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, 
                                                  Y_train, 
                                                  test_size=0.25, 
                                                  random_state=123)

# Compile and train the model to measure validation accuracy.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=128, nb_epoch=2,
                    verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % (X_train.shape[0] / float(X_val.shape[0]))
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 3s - loss: 0.7374 - acc: 0.7994 - val_loss: 0.4424 - val_acc: 0.8849
Epoch 2/2
29406/29406 [==============================] - 3s - loss: 0.6734 - acc: 0.8249 - val_loss: 0.5455 - val_acc: 0.8555


**Validation Accuracy**: 0.88

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [3]:
# Re-construct the network and add a convolutional layer before 
# the first fully-connected layer.
import pickle
import numpy as np

from keras.layers import Convolution2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

with open('train.p', 'rb') as f:
    train = pickle.load(f)
with open('test.p', 'rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'].astype('float32'), train['labels'].astype('float32')
X_test, y_test = test['features'].astype('float32'), test['labels'].astype('float32')

X_train /= 255.0
X_train -= 0.5

Y_train = np_utils.to_categorical(y_train, 43)
Y_test = np_utils.to_categorical(y_test, 43)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  Y_train, 
                                                  test_size=0.25, 
                                                  random_state=123)

model = Sequential()

model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(3072, name='input'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(3072, name='hidden1'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(43, name='output'))
model.add(Activation('softmax'))


# Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

/home/mimoralea/anaconda3/envs/keras/lib/python3.5/site-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 15s - loss: 1.1762 - acc: 0.6686 - val_loss: 0.2826 - val_acc: 0.9204
Epoch 2/2
29406/29406 [==============================] - 14s - loss: 0.2092 - acc: 0.9376 - val_loss: 0.1555 - val_acc: 0.9528


**Validation Accuracy**: 0.95

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [6]:
# Re-construct the network and add a pooling layer after the convolutional layer.
import pickle
import numpy as np

from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

with open('train.p', 'rb') as f:
    train = pickle.load(f)
with open('test.p', 'rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'].astype('float32'), train['labels'].astype('float32')
X_test, y_test = test['features'].astype('float32'), test['labels'].astype('float32')

X_train /= 255.0
X_train -= 0.5

Y_train = np_utils.to_categorical(y_train, 43)
Y_test = np_utils.to_categorical(y_test, 43)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  Y_train, 
                                                  test_size=0.25, 
                                                  random_state=123)

model = Sequential()

model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(3072, name='input'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(3072, name='hidden1'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(43, name='output'))
model.add(Activation('softmax'))


# Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

/home/mimoralea/anaconda3/envs/keras/lib/python3.5/site-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 5s - loss: 1.0924 - acc: 0.6821 - val_loss: 0.2688 - val_acc: 0.9194
Epoch 2/2
29406/29406 [==============================] - 5s - loss: 0.2062 - acc: 0.9414 - val_loss: 0.1727 - val_acc: 0.9539


**Validation Accuracy**: 0.95

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [7]:
# Re-construct the network and add dropout after the pooling layer.
import pickle
import numpy as np

from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

with open('train.p', 'rb') as f:
    train = pickle.load(f)
with open('test.p', 'rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'].astype('float32'), train['labels'].astype('float32')
X_test, y_test = test['features'].astype('float32'), test['labels'].astype('float32')

X_train /= 255.0
X_train -= 0.5

Y_train = np_utils.to_categorical(y_train, 43)
Y_test = np_utils.to_categorical(y_test, 43)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  Y_train, 
                                                  test_size=0.25, 
                                                  random_state=123)

model = Sequential()

model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.50))

model.add(Flatten())
model.add(Dense(3072, name='input'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(3072, name='hidden1'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(43, name='output'))
model.add(Activation('softmax'))


# Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

/home/mimoralea/anaconda3/envs/keras/lib/python3.5/site-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 5s - loss: 1.1148 - acc: 0.6693 - val_loss: 0.2871 - val_acc: 0.9215
Epoch 2/2
29406/29406 [==============================] - 5s - loss: 0.2768 - acc: 0.9176 - val_loss: 0.1761 - val_acc: 0.9566


**Validation Accuracy**: 0.95

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [11]:
import pickle
import numpy as np

from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

with open('train.p', 'rb') as f:
    train = pickle.load(f)
with open('test.p', 'rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'].astype('float32'), train['labels'].astype('float32')
X_test, y_test = test['features'].astype('float32'), test['labels'].astype('float32')

X_train /= 255.0
X_train -= 0.5

Y_train = np_utils.to_categorical(y_train, 43)
Y_test = np_utils.to_categorical(y_test, 43)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  Y_train, 
                                                  test_size=0.25, 
                                                  random_state=123)

model = Sequential()

model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.50))

model.add(Flatten())
model.add(Dense(3072, name='input'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(3072, name='hidden1'))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(43, name='output'))
model.add(Activation('softmax'))


# Compile and train the model.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=5,
                    verbose=1, validation_data=(X_val, y_val))

/home/mimoralea/anaconda3/envs/keras/lib/python3.5/site-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


Train on 29406 samples, validate on 9803 samples
Epoch 1/5
29406/29406 [==============================] - 6s - loss: 1.0376 - acc: 0.6967 - val_loss: 0.1555 - val_acc: 0.9614
Epoch 2/5
29406/29406 [==============================] - 5s - loss: 0.1564 - acc: 0.9524 - val_loss: 0.0867 - val_acc: 0.9749
Epoch 3/5
29406/29406 [==============================] - 5s - loss: 0.0817 - acc: 0.9747 - val_loss: 0.0536 - val_acc: 0.9841
Epoch 4/5
29406/29406 [==============================] - 5s - loss: 0.0640 - acc: 0.9807 - val_loss: 0.0356 - val_acc: 0.9907
Epoch 5/5
29406/29406 [==============================] - 5s - loss: 0.0477 - acc: 0.9851 - val_loss: 0.0393 - val_acc: 0.9894


**Best Validation Accuracy:** 0.98

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [15]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.201123195168
Test accuracy: 0.946872525751


**Test Accuracy:** 0.95

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.

Indeed!!!!